In [1]:
with open("testday12.txt") as f:
    data = f.read().splitlines()

# Part 1

## The person
Define a Person Class, this will make a person object that is used to track the position of the Person walking the map. Additionally it will count the moves the person takes to reach its goal.

Define a Location Class, this will give each location that is visited on the map a state. before you move forward, a location will be made of your current location, containing:
- the x and y coords,
- the path you took to get there, 
- the return_path (showing both the path to get there and the directions already explored are blocked)
- the local value (a-z)
- the iteration count with which it was last updated

In [2]:
class Person:
    def __init__(self, person_posx, person_posy, location_val, steps):
        self.posx = person_posx
        self.posy = person_posy 
        self.val = location_val
        self.steps = steps

In [3]:
class Location:
    def __init__(self, locx, locy, pathstate, return_pathstate, locval, move_):
        self.posx = locx
        self.posy = locy
        self.path = pathstate.copy()
        self.return_path = return_pathstate.copy()
        self.locval = locval
        self.move = move_

## The Lookaround
The lookaround will be used to see the values around the current position. If the current position is at the edge, the returned value outside the grid wil be None

In [4]:
def lookaround(posx, posy):
    maxx = len(data[0])-1
    maxy = len(data)-1
    
    # Make left and right values:
    if posx == 0:
        left_val = None
        right_val = last_visited[-1].path[posy][posx+1]
    elif posx == maxx:
        left_val = last_visited[-1].path[posy][posx-1]
        right_val = None
    elif posx> 0 and posx < maxx:
        left_val = last_visited[-1].path[posy][posx-1]
        right_val = last_visited[-1].path[posy][posx+1]

    # make up and down values:
    if posy == 0:
        up_val = None
        down_val = last_visited[-1].path[posy+1][posx]
    elif posy == maxy:
        up_val = last_visited[-1].path[posy-1][posx]
        down_val = None
    else:
        up_val = last_visited[-1].path[posy-1][posx]
        down_val = last_visited[-1].path[posy+1][posx]
    
    positions = [left_val, right_val, up_val, down_val]
    return positions

return_lookaround will do the same as look around, but base the lookaround on the return_path value. This ensures that the program will not run in to the same path over and over again

In [5]:
def return_lookaround(posx, posy):
    maxx = len(data[0])-1
    maxy = len(data)-1
    
    # Make left and right values:
    if posx == 0:
        left_val = None
        right_val = last_visited[-1].return_path[posy][posx+1]
    elif posx == maxx:
        left_val = last_visited[-1].return_path[posy][posx-1]
        right_val = None
    elif posx> 0 and posx < maxx:
        left_val = last_visited[-1].return_path[posy][posx-1]
        right_val = last_visited[-1].return_path[posy][posx+1]

    # make up and down values:
    if posy == 0:
        up_val = None
        down_val = last_visited[-1].return_path[posy+1][posx]
    elif posy == maxy:
        up_val = last_visited[-1].return_path[posy-1][posx]
        down_val = None
    else:
        up_val = last_visited[-1].return_path[posy-1][posx]
        down_val = last_visited[-1].return_path[posy+1][posx]
    
    positions = [left_val, right_val, up_val, down_val]
    return positions

## The Checker
the checker will check if the values are possible moves. it will returtn a dictoniary with a false or true value for each direction.

In [6]:
def checker(positions_):
    available = {}
    sides = ["left", "right", "up", "down"]
    for i, pos in enumerate(positions_):
        if pos == None:
            available[sides[i]] = False
            continue
           
        posval = ord(pos) - 96
        myval = ord(my.val) - 96
        dif = posval - myval
        if dif <= 1:
            available[sides[i]] = True
        else:
            available[sides[i]] = False
            
    return available

Mover will make a new Location object with: 
- the current x and y coords, 
- the current path the person has taken,
- the current path will also function as returned path, as you can not move into an already visited spot, so there should be no return values during the first time entering the spot.
- the local height value
- the iteration count

Afterwards it will update the persons x and y coords to the new position, and add a step to the stepcount.

In [7]:
def mover(move_to, move):
    locval = my.val
    stop = "~"
    path[my.posy] = path[my.posy][:my.posx] + stop + path[my.posy][my.posx+1:]
    
    last_visited.append(Location(my.posx, my.posy, path, path, locval, move))
    
    if move_to == "left":
        my.posx -= 1
    if move_to == "right":
        my.posx += 1
    if move_to == "up":
        my.posy -= 1
    if move_to == "down":
        my.posy += 1
        
    my.val = path[my.posy][my.posx]
    my.steps += 1
    

Returner will check if there are possible directions to move in. If there are directions to move in, skip this actions. If there are NO directions to move in:

- update x and y coords of my.Person
- reduces steps by 1
- update my.Persons height value
- delete the latest last_visited location object (As we returned to the state before.)
- update the return path to block the direction we just came from.

In [8]:
def returner(options, path):
    movecheck = 0
    for option in options:
        if options[option] == True:
            movecheck += 1
            
    if movecheck == 0:         
        path = last_visited[-2].path
        return_path = last_visited[-2].return_path
        stop = "~"
        return_path[my.posy] = return_path[my.posy][:my.posx] + stop + return_path[my.posy][my.posx+1:]

        my.steps -= 1
        my.posx = last_visited[-1].posx
        my.posy = last_visited[-1].posy
        my.val = last_visited[-1].locval
        last_visited.pop()
        last_visited[-1].path = path.copy()
        last_visited[-1].return_path = return_path.copy()
        return path, True
    return path, False

Endreturner, does the same as returner but twice, and is only called when the end has been reached.
(I think this one is redundant but was necessary at one point to avoid the end point value being changed to "~".)

In [9]:
def endreturner():
    path = last_visited[-3].path
    stop = "~"
    path[last_visited[-1].posy] = path[last_visited[-1].posy][:last_visited[-1].posx] + stop + path[last_visited[-1].posy][last_visited[-1].posx+1:]
    my.steps -= 2
    my.posx = last_visited[-2].posx
    my.posy = last_visited[-2].posy
    my.val = last_visited[-2].locval
    last_visited.pop()
    last_visited.pop()
    last_visited[-1].path = path.copy()
    
    return path

## Start Position
The start position will be found by a regex function iterating over the input data. This will be used as input for the person.

In [10]:
import re

for i, line in enumerate(data):
    start = re.search(r"S", line)
    if bool(start) == True:
        starty = i
        startx = start.span(0)[0]
        
print(startx, starty)

0 0


In [11]:
for i, line in enumerate(data):
    start = re.search(r"E", line)
    if bool(start) == True:
        endy = i
        endx = start.span(0)[0]
print(endx, endy)

5 2


## Test weird stuff

In [12]:
### Create a person, this will keep track of where in the field we are, and what our current value is.
my = Person(startx, starty, "s", 0)

### Create a path that will be modified based on what routes we have already encountered.
path = data.copy()
end = "z"
path[endy] = path[endy][:endx] + end + path[endy][endx+1:]

### Create 2 lists: after each succesfull run to the finish, runs will be updated with the amount of steps it took to complete. 
### Last visited will contain objects that remember the state of the path as we move forwards or backwards.
runs = [500]
last_visited = []

### Makes the program first move forward
return_state = False

### Create 2 locations so that the you can return to a start position without going out of range.
last_visited.append(Location(my.posx, my.posy, path, path, my.val, 0))
last_visited.append(Location(my.posx, my.posy, path, path, my.val, 0))

### This block of code will loop through all moves, each move will consist of the following steps:
### - Check if we are at the end?
### - Look around the current position to see what height the next blocks are
### - Check which blocks are currently available to step on.

### - If there are no possible steps, return to the previous state of the path and block the location we come from.
### - If there are possible moves: move to the next block and block the way behind.

for move in range(1,5000):
    
    if len(last_visited) == 1:
        print("ended run in " + str(move) + " moves.")
        break
    
    #Check if at end of run?
    if my.posx == endx and my.posy == endy:
        print("Done! Took me " + str(my.steps) + " steps.")
        runs.append(my.steps)
        path = endreturner()
    
    #Check the positions around current location: 
    #if return_state == True it checks based on the returned_path (blocks the path that it returned from)
    if return_state == False:
        positions = lookaround(my.posx, my.posy)
    elif return_state == True:
        positions = return_lookaround(my.posx, my.posy)
        
    #Check the possible options
    options = checker(positions)    
    
    #print the path it has currently taken
    print("path currently taken:")
    for p in last_visited[-1].path:
        print(p) 
    print("path currently blocked:")
    for p in last_visited[-1].return_path:
        print(p) 
    #print the current x and y coords, current value, current values around it, and their possibilities to take the path
    print("x="+str(my.posx) + " y="+str(my.posy))
    print("my value:", my.val)
    print("values LRUD:", positions)
    print("options LRUD:", options)
    print("iteration " + str(move), "objects in mem: ", len(last_visited))  
    print("\n")    
    
    #return to earlier path if step amount exceeds earlier run completion
    if my.steps > min(runs):
        for option in options:
            options[option] = False
            
    #Check returner (if move not possible, return to a previous step and block the path it previously took)
    path, return_ = returner(options, path)
    
    #Check if you are back at the start, with no way ahead. if both True, print that run has ended and break run.
    stopcheck = 0
    for option in options:
        if options[option] == True:
            stopcheck += 1

    #Mark the current location as last location, move my.Person to new location and block the way back
    if return_ == False:
        for option in options:
            if options[option] == True:
                move_to = option
                mover(move_to, move)
                break
    
    #Set return state to decide which poisition checker to use.
    return_state = return_          

path currently taken:
Sabqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
path currently blocked:
Sabqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
x=0 y=0
my value: s
values LRUD: [None, 'a', None, 'a']
options LRUD: {'left': False, 'right': True, 'up': False, 'down': True}
iteration 1 objects in mem:  2


path currently taken:
~abqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
path currently blocked:
~abqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
x=1 y=0
my value: a
values LRUD: ['~', 'b', None, 'b']
options LRUD: {'left': False, 'right': True, 'up': False, 'down': True}
iteration 2 objects in mem:  3


path currently taken:
~~bqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
path currently blocked:
~~bqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
x=2 y=0
my value: b
values LRUD: ['~', 'q', None, 'c']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': True}
iteration 3 objects in mem:  4


path currently taken:
~~~qponm
abcryxxl
accszzxk
acctuvwj
abdefghi
path currently blocked:
~~~qponm
abcryxxl
a

path currently taken:
~~~~~~~~
a~~~~~~~
~~c~zz~~
~~~~~~~~
~~~~~~~~
path currently blocked:
~~~~~~~~
a~~~~~~~
~~c~zz~~
~~~~~~~~
~~~~~~~~
x=4 y=2
my value: z
values LRUD: ['~', 'z', '~', '~']
options LRUD: {'left': False, 'right': True, 'up': False, 'down': False}
iteration 171 objects in mem:  38


Done! Took me 37 steps.
path currently taken:
~~~~~~~~
a~~~y~~~
~~c~~z~~
~~~~~~~~
~~~~~~~~
path currently blocked:
~~~~~~~~
a~~~y~~~
~~c~zz~~
~~~~~~~~
~~~~~~~~
x=4 y=1
my value: y
values LRUD: ['~', '~', '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 172 objects in mem:  37


path currently taken:
~~~~~~~~
a~~~yx~~
~~c~zz~~
~~~~~~~~
~~~~~~~~
path currently blocked:
~~~~~~~~
a~~~~x~~
~~c~zz~~
~~~~~~~~
~~~~~~~~
x=5 y=1
my value: x
values LRUD: ['~', '~', '~', 'z']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 173 objects in mem:  36


path currently taken:
~~~~~~~~
a~~~yxx~
~~c~zz~~
~~~~~~~~
~~~~~~~~
path curre

a~~szzxk
ac~tuvwj
ab~efghi
path currently blocked:
~~~qponm
a~~ryxxl
a~~szzxk
ac~tuvwj
~b~efghi
x=1 y=4
my value: b
values LRUD: ['~', '~', 'c', None]
options LRUD: {'left': False, 'right': False, 'up': True, 'down': False}
iteration 236 objects in mem:  11


path currently taken:
~~~qponm
a~~ryxxl
a~~szzxk
ac~tuvwj
a~~efghi
path currently blocked:
~~~qponm
a~~ryxxl
a~~szzxk
ac~tuvwj
a~~efghi
x=1 y=3
my value: c
values LRUD: ['a', '~', '~', '~']
options LRUD: {'left': True, 'right': False, 'up': False, 'down': False}
iteration 237 objects in mem:  12


path currently taken:
~~~qponm
a~~ryxxl
a~~szzxk
a~~tuvwj
a~~efghi
path currently blocked:
~~~qponm
a~~ryxxl
a~~szzxk
a~~tuvwj
a~~efghi
x=0 y=3
my value: a
values LRUD: [None, '~', 'a', 'a']
options LRUD: {'left': False, 'right': False, 'up': True, 'down': True}
iteration 238 objects in mem:  13


path currently taken:
~~~qponm
a~~ryxxl
a~~szzxk
~~~tuvwj
a~~efghi
path currently blocked:
~~~qponm
a~~ryxxl
a~~szzxk
~~~tuvwj
a~~efghi
x=0 y=

iteration 409 objects in mem:  22


path currently taken:
~~~qponm
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~qponm
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=7 y=0
my value: m
values LRUD: ['n', None, None, '~']
options LRUD: {'left': True, 'right': False, 'up': False, 'down': False}
iteration 410 objects in mem:  23


path currently taken:
~~~qpon~
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~qpon~
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=6 y=0
my value: n
values LRUD: ['o', '~', None, 'x']
options LRUD: {'left': True, 'right': False, 'up': False, 'down': False}
iteration 411 objects in mem:  24


path currently taken:
~~~qpo~~
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~qpo~~
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=5 y=0
my value: o
values LRUD: ['p', '~', None, 'x']
options LRUD: {'left': True, 'right': False, 'up': False, 'down': False}
iteration 412 objects in mem:  25


path currently taken:
~~~qp~~~
ab~ryxx~
~~~szzx~
~~~tuvw~
~~~~~

~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~~~~~~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=3 y=1
my value: r
values LRUD: ['~', 'y', '~', 's']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': True}
iteration 475 objects in mem:  30


path currently taken:
~~~~~~~~
~~~~yxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~~~~~~
~~~~yxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=3 y=2
my value: s
values LRUD: ['~', 'z', '~', 't']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': True}
iteration 476 objects in mem:  31


path currently taken:
~~~~~~~~
~~~~yxx~
~~~~zzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~~~~~~~
~~~~yxx~
~~~~zzx~
~~~tuvw~
~~~~~~~~
x=3 y=3
my value: t
values LRUD: ['~', 'u', '~', '~']
options LRUD: {'left': False, 'right': True, 'up': False, 'down': False}
iteration 477 objects in mem:  32


path currently taken:
~~~~~~~~
~~~~yxx~
~~~~zzx~
~~~~uvw~
~~~~~~~~
path currently blocked:
~~~~~~~~
~~~~yxx~
~~~~zzx~
~~~~uvw~
~~~~~~~

...

path currently taken:
~abq~~~~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
path currently blocked:
~a~q~~~~
~~c~yxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
x=3 y=0
my value: q
values LRUD: ['~', '~', None, '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2170 objects in mem:  21


path currently taken:
~abqp~~~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
path currently blocked:
~ab~p~~~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
x=4 y=0
my value: p
values LRUD: ['~', '~', None, 'y']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2171 objects in mem:  20


path currently taken:
~abqpo~~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
path currently blocked:
~abq~o~~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
x=5 y=0
my value: o
values LRUD: ['~', '~', None, 'x']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2172 objects in mem:  19


path currently taken:
~abqpon~
~~cryxx~
a~~szzx~
ac~tuvw~
ab~~~~~~
path currently blocked:
~ab

iteration 2343 objects in mem:  12


path currently taken:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~ghi
path currently blocked:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~ghi
x=5 y=4
my value: g
values LRUD: ['~', 'h', 'v', None]
options LRUD: {'left': False, 'right': True, 'up': False, 'down': False}
iteration 2344 objects in mem:  13


path currently taken:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~~hi
path currently blocked:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~~hi
x=6 y=4
my value: h
values LRUD: ['~', 'i', 'w', None]
options LRUD: {'left': False, 'right': True, 'up': False, 'down': False}
iteration 2345 objects in mem:  14


path currently taken:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~~~i
path currently blocked:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~~~~~~~i
x=7 y=4
my value: i
values LRUD: ['~', None, 'j', None]
options LRUD: {'left': False, 'right': False, 'up': True, 'down': False}
iteration 2346 objects in mem:  15


path currently taken:
~abqponm
~bcryxxl
~ccszzxk
~~~tuvwj
~

iteration 2398 objects in mem:  27


path currently taken:
~ab~~~~~
~~~~yxx~
~~cszzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~ab~~~~~
~~~~yxx~
~~cszzx~
~~~tuvw~
~~~~~~~~
x=2 y=2
my value: c
values LRUD: ['~', 's', '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2399 objects in mem:  28


path currently taken:
~ab~~~~~
~~~~yxx~
~ccszzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~ab~~~~~
~~~~yxx~
~c~szzx~
~~~tuvw~
~~~~~~~~
x=1 y=2
my value: c
values LRUD: ['~', '~', '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2400 objects in mem:  27


path currently taken:
~ab~~~~~
~b~~yxx~
~ccszzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~~b~~~~~
~b~~yxx~
~~cszzx~
~~~tuvw~
~~~~~~~~
x=1 y=1
my value: b
values LRUD: ['~', '~', '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2401 objects in mem:  26


path currently taken:
~ab~~~~~
~bc~yxx~
~ccszzx~
~~~tuvw~
~~

~~~szzxk
~~~tuvw~
~~~~~~~~
path currently blocked:
~abqponm
~bcryxx~
~~~szzxk
~~~tuvw~
~~~~~~~~
x=7 y=2
my value: k
values LRUD: ['x', None, '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2578 objects in mem:  19


path currently taken:
~abqponm
~bcryxxl
~~~szzxk
~~~tuvwj
~~~~~~~~
path currently blocked:
~abqponm
~bcryxxl
~~~szzx~
~~~tuvwj
~~~~~~~~
x=7 y=3
my value: j
values LRUD: ['w', None, '~', '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2579 objects in mem:  18


path currently taken:
~abqponm
~bcryxxl
~~~szzxk
~~~tuvwj
~~~~~~~i
path currently blocked:
~abqponm
~bcryxxl
~~~szzxk
~~~tuvw~
~~~~~~~i
x=7 y=4
my value: i
values LRUD: ['~', None, '~', None]
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2580 objects in mem:  17


path currently taken:
~abqponm
~bcryxxl
~~~szzxk
~~~tuvwj
~~~~~~hi
path currently blocked:
~abqponm
~bcryxxl
~~~szzxk
~~~tuvwj
~~~~~~

~~~tuvw~
~~~~~~~~
path currently blocked:
~a~q~~~~
~~~~yxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=3 y=0
my value: q
values LRUD: ['~', '~', None, '~']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2628 objects in mem:  27


path currently taken:
~abqp~~~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~ab~p~~~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=4 y=0
my value: p
values LRUD: ['~', '~', None, 'y']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2629 objects in mem:  26


path currently taken:
~abqpo~~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~abq~o~~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=5 y=0
my value: o
values LRUD: ['~', '~', None, 'x']
options LRUD: {'left': False, 'right': False, 'up': False, 'down': False}
iteration 2630 objects in mem:  25


path currently taken:
~abqpon~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
path currently blocked:
~abqp~n~
~~~ryxx~
~~~szzx~
~~~tuvw~
~~~~~~~~
x=6 y=0

In [13]:
last_visited

In [14]:
runs

[500, 37, 37, 33, 33, 33, 31, 31, 31, 31, 31, 31, 31]

In [15]:
for p in last_visited[-1].path:
    print(p)

Sabqponm
abcryxxl
accszzxk
acctuvwj
abdefghi


In [16]:
for p in last_visited[-1].return_path:
    print(p)

~abqponm
abcryxxl
accszzxk
acctuvwj
abdefghi
